<a href="https://colab.research.google.com/github/YapingWu/GoogleColab/blob/main/lstm/generate_text_with_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考资料：[python深度学习之LSTM生成文本](https://clownote.github.io/2020/08/20/DeepLearningWithPython/Deep-Learning%20with-Python-ch8_1/)

# 数据准备

In [1]:
# 下载语料，并将其转换为全小写

import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt', 
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with open(path, 'r') as f:
  text = f.read().lower()

print('Corpus length:', len(text))

606208/600901 [==============================] - 0s 0us/step
Corpus length: 600893


In [2]:
# 将字符序列向量化

maxlen = 60     # 每个序列的长度
step = 3        # 每 3 个字符采样一个新序列
sentences = []  # 保存所提取的序列
next_chars = [] # sentences 的下一个字符

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i+maxlen])
    next_chars.append(text[i+maxlen])
print('Number of sequences:', len(sentences))

chars = sorted(list(set(text)))
char_indices = dict((char, chars.index(char)) for char in chars)
# 插：上面这两行代码 6
print('Unique characters:', len(chars))

print('Vectorization...')

x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200278
Unique characters: 57
Vectorization...


# 构建网络
我们要用到的网络其实很简单，一个 LSTM 层 + 一个 softmax 激活的 Dense 层就可以了。其实并不一定要用 LSTM，用一维卷积层也是可以生成序列的。

In [3]:
# 用于预测下一个字符的单层 LSTM 模型

from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

In [4]:
# 模型编译配置

from keras import optimizers

optimizer = optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer)

# 训练语言模型并从中采样
给定一个语言模型和一个种子文本片段，就可以通过重复以下操作来生成新的文本：

给定目前已有文本，从模型中得到下一个字符的概率分布；  
根据某个温度对分布进行重新加权；  
根据重新加权后的分布对下一个字符进行随机采样；  
将新字符添加到文本末尾。  

In [5]:
# 采样函数
def sample(preds, temperature=1.0):
    '''
    对模型得到的原始概率分布重新加权，并从中抽取一个字符索引
    '''
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
# 文本生成循环

import random

for epoch in range(1, 60):    # 训练 60 个轮次
    print(f'👉\033[1;35m epoch {epoch} \033[0m')    # print('epoch', epoch)
    
    model.fit(x, y,
              batch_size=128,
              epochs=1)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print(f'  📖 Generating with seed: "\033[1;32;43m{generated_text}\033[0m"')    # print(f' Generating with seed: "{generated_text}"')
    
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print(f'\n   \033[1;36m 🌡️ temperature: {temperature}\033[0m')    # print('\n  temperature:', temperature)
        print(generated_text, end='')
        for i in range(400):    # 生成 400 个字符
            # one-hot 编码目前有的文本
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1
            
            # 预测，采样，生成下一字符
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            print(next_char, end='')
            
            generated_text = generated_text[1:] + next_char
            
    print('\n' + '-' * 20)

👉 epoch 1 
1565/1565 [==============================] - 296s 188ms/step - loss: 2.2684
  📖 Generating with seed: "ith all sufferers"--and
suffering itself is looked upon by t"

    🌡️ temperature: 0.2
ith all sufferers"--and
suffering itself is looked upon by the subjent of the stand of the souls of the stance of the states that the soul of the states of the state of the supposing to the some the still and the something to the soul and sense of the states of the so has to the stance of the soulted and so has one and souls of the something the something and every so ground of the souls of the strenger the soulted the sense of the such and soul and souls 
    🌡️ temperature: 0.5
renger the soulted the sense of the such and soul and souls to he deather that it is every precessions of the not the such a more on the philosopher of the now propesting to expecience, one present of not in the
sense of so a proment of the renountly and feeling of the consence of here of the men, and should pres